In [3]:
from pathlib import Path
# actual_path = '/Users/danishkarur/opt/Python Scripts/Learnings_CognitiveClasses/Spark/LabData'
for path in Path('LabData').iterdir():
    print(path)

LabData/followers.txt
LabData/.DS_Store
LabData/notebook.log
LabData/nycweather.csv
LabData/pom.xml
LabData/README.md
LabData/nyctaxi100.csv
LabData/users.txt
LabData/nyctaxisub.csv
LabData/taxistreams.py
LabData/nyctaxi.csv


In [4]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext.getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/20 20:45:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [5]:
sc.version

'3.4.0'

In [6]:
readme = sc.textFile('LabData/README.md')
readme.count()

98

In [7]:
readme.first()

'# Apache Spark'

In [8]:
linesWithSpark = readme.filter(lambda line:'Spark' in line)
print(linesWithSpark.count())
# readme.filter(lambda line: "Spark" in line).count()

18


More on RDD Operations

This section builds upon the previous section. In this section, you will see that RDD can be used for more complex computations. You will find the line from that "README.md" file with the most words in it.

Run the following cell.

In [9]:
readme.map(lambda line:len(line.split())).reduce(lambda a,b: a if (a>b) else b)

# There are two parts to this. The first maps a line to an integer value, 
# the number of words in that line. 
# In the second part reduce is called to find the line with the most words in it. 
# The arguments to map and reduce are Python anonymous functions (lambdas), 
# but you can use any top level Python functions. In the next step, 
# you’ll define a max function to illustrate this feature.

14

In [10]:
# Now define the max_ function

def my_max(a, b):
    if a > b:
        return a
    else:
        return b
    


In [11]:
readme.map(lambda line:len(line.split())).reduce(my_max)

14

In [14]:
WordsCount = readme.flatMap(lambda line:line.split()).map(lambda word:(word,1)).reduceByKey(lambda a,b: a+b)
WordsCount.count()

266

Here we combined the flatMap, map, and the reduceByKey functions to do a word count of each word in the readme file.

To collect the word counts, use the collect action.

It should be noted that the collect function brings all of the data into the driver node. For a small dataset, 
this is acceptable but, for a large dataset this can cause an Out Of Memory error. 
It is recommended to use collect() for testing only. The safer approach is to use the take() function e.g. print take(n)

In [15]:
WordsCount.collect()

[('#', 1),
 ('Apache', 1),
 ('Spark', 14),
 ('is', 6),
 ('It', 2),
 ('provides', 1),
 ('high-level', 1),
 ('APIs', 1),
 ('in', 5),
 ('Scala,', 1),
 ('Java,', 1),
 ('an', 3),
 ('optimized', 1),
 ('engine', 1),
 ('supports', 2),
 ('computation', 1),
 ('analysis.', 1),
 ('set', 2),
 ('of', 5),
 ('tools', 1),
 ('SQL', 2),
 ('MLlib', 1),
 ('machine', 1),
 ('learning,', 1),
 ('GraphX', 1),
 ('graph', 1),
 ('processing,', 1),
 ('Documentation', 1),
 ('latest', 1),
 ('programming', 1),
 ('guide,', 1),
 ('[project', 2),
 ('README', 1),
 ('only', 1),
 ('basic', 1),
 ('instructions.', 1),
 ('Building', 1),
 ('using', 2),
 ('[Apache', 1),
 ('run:', 1),
 ('do', 2),
 ('this', 1),
 ('downloaded', 1),
 ('documentation', 3),
 ('project', 1),
 ('site,', 1),
 ('at', 2),
 ('Spark"](http://spark.apache.org/docs/latest/building-spark.html).', 1),
 ('Interactive', 2),
 ('Shell', 2),
 ('The', 1),
 ('way', 1),
 ('start', 1),
 ('Try', 1),
 ('following', 2),
 ('1000:', 2),
 ('scala>', 1),
 ('1000).count()', 1),


In [16]:
# Most frequent word in this README file

WordsCount.reduce(lambda a,b: a if a[1]>b[1] else b)

('the', 21)

Using Spark caching

In this short section, you’ll see how Spark caching can be 
used to pull data sets into a cluster-wide in-memory cache. 
This is very useful for accessing repeated data, such as querying a small “hot” dataset or 
when running an iterative algorithm. Both Python and Scala use the same commands.

As a simple example, let’s mark our linesWithSpark dataset to be cached and 
then invoke the first count operation to tell Spark to cache it. 
Remember that transformation operations such as cache does not get processed until some action like count() is called. 
Once you run the second count() operation, 
you should notice a small increase in speed

In [17]:
print(linesWithSpark.count())

18


In [18]:
from timeit import Timer
def count():
    return linesWithSpark.count()
t = Timer(lambda: count())

In [19]:
print(t.timeit(number=50))

0.9611677909999798


In [20]:
linesWithSpark.cache()
print(t.timeit(number=50))

0.7766033330000255


It may seem silly to cache such a small file, 
but for larger data sets across tens or hundreds of nodes, 
this would still work. 
The second linesWithSpark.count() action runs against 
the cache and would perform significantly better for large datasets.